# Preparations

Import various libraries: Mainly Tensorflow & Keras

> **TODO: Remove unnecessary imports**

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D

Using TensorFlow backend.


Setting some general values.

In [2]:
IMG_HEIGHT = 200 # original is 300
IMG_WIDTH = 200

BATCH_SIZE = 10

# Load the Images

First we load the classified images.

In [3]:
dataset_dir = './dataset/'

img_gen = ImageDataGenerator(
    rescale = 1./255, # Rescale [0, 255] to [0, 1] because NN prefer smaller input
    horizontal_flip = True, # Randomly flip inputs horizontally
    vertical_flip = True, # Randomly flips inputs vertically
    height_shift_range = .2,
    validation_split = 0.2 # mark  20% as belonging to the validation set
)

train_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'training'
)

val_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'validation'
)


Found 1751 images belonging to 3 classes.
Found 437 images belonging to 3 classes.


We have 2188 images in total.
They have now been split up in training and validation sets (that are strictly not overlapping)

# Create the model

This is the part that has to be optimized the most! (and isn't at all yet)

In [4]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='sigmoid')
])

# Compile the model

In [5]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40000)            

In [6]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=BATCH_SIZE,
    epochs=5,
    validation_data=val_gen,
    validation_steps=BATCH_SIZE
)

Epoch 1/5
10/10 [==============================] - 12s 1s/step - loss: 0.8058 - accuracy: 0.5733 - val_loss: 0.6374 - val_accuracy: 0.6667
Epoch 2/5
10/10 [==============================] - 7s 668ms/step - loss: 0.6309 - accuracy: 0.6567 - val_loss: 0.6107 - val_accuracy: 0.6667
Epoch 3/5
10/10 [==============================] - 7s 692ms/step - loss: 0.6152 - accuracy: 0.6767 - val_loss: 0.7022 - val_accuracy: 0.6433
Epoch 4/5
10/10 [==============================] - 7s 652ms/step - loss: 0.5920 - accuracy: 0.6800 - val_loss: 0.5516 - val_accuracy: 0.6900
Epoch 5/5
10/10 [==============================] - 7s 679ms/step - loss: 0.6061 - accuracy: 0.6600 - val_loss: 0.5593 - val_accuracy: 0.7045
